**Descripción de la Actividad Evaluativa**

Vamos a crear un sistema de facturación y bodega simplificado, el cual
tendrá diferentes opciones:

a) Ingresar producto: debe ingresarse el nombre y el precio del producto.
Tendrá un sub menú: nuevo producto, guardar archivo o volver.

b) Ingresar factura: debe ingresarse el nombre del cliente, luego deberá
generar un submenú: ingresar producto, guardar y salir.

Los productos se ingresan con nombre y precio. La factura además
debe guardar la fecha y hora actual de la venta.

c) Ver resumen: en esta opción se debe desplegar:
Cantidad de ventas y monto total vendido. Opción de guardar
resumen (ingresando nombre de archivo).

d) Ver detalle: se debe tener las opciones desplegar: ventas por usuario,
ventas por producto, ventas diarias, guardar vista (ingresando el
nombre de archivo).

e) Borrar datos: elimina los archivos guardados (elegir productos,
facturas, todos o volver)

f) Salir: termina la ejecución del programa

Cabe señalar, que, si los archivos ya existen, entonces deben ser cargados
al inicio.

Imports

In [16]:
import pandas as pd
# pandas para trabajar con datos
from datetime import datetime
# datetime para trabajar con fechas
import re
# re para expresines regulares

Funciones Para el ingreso por teclado

In [17]:
# Aqui se valida que se ingresen numeros
def ingresaFloat(mensaje):
  while True:
    try:
      return float(input(mensaje))
    except ValueError:
        print("Ingresa un Numero Valido !")

# Aqui se valida que se ingresen numeros
def ingresaEntero(mensaje):
  while True:
    try:
      return int(input(mensaje))
    except ValueError:
        print("Ingresa un Numero Valido !")


# Aqui se pone una expresion regular para validar el texto
def ingresaTexto(mensaje):
  patron = r'^[a-zA-Z\s]+$'
  while True:
    try:
      texto = input(mensaje)
      if re.match(patron, texto):
        return texto
      else:
        raise ValueError
    except ValueError:
      print("Ingresa un Texto Valido!")

# Aqui se pone una expresion regular para validar el texto con numeros
def ingresaTexto_con_Numeros(mensaje):
  patron = r'^[a-zA-Z0-9\s]+$'
  while True:
    try:
      texto = input(mensaje)
      if re.match(patron, texto):
        return texto
      else:
        raise ValueError
    except ValueError:
      print("Ingresa un Texto Valido!")

Variables para los nombres de archivos

In [18]:
# Aqui se definen los nombres de los archivos que se usaran mas adelante
archivo_productos='productos.csv'
archivo_facturas='facturas.csv'
archivo_resumen='resumen.csv'
archivo_detalle='detalle.csv'

Lectura o creacion de archivos

In [19]:
# Aqui se leen los archivos o se crean si no existen
try:
      df_productos= pd.read_csv(archivo_productos)
except FileNotFoundError:
      df_productos = pd.DataFrame(columns=['Nombre', 'Precio'])

try:
      df_facturas = pd.read_csv(archivo_facturas)
except FileNotFoundError:
      df_facturas = pd.DataFrame(columns=['Cliente', 'Fecha', 'Productos'])


Funciones

In [24]:
# Aqui se ingresan los productos al archivo csv
def ingresar_producto():
  global df_productos
  print(f"\nIngreso de Producto:")
  nombre = ingresaTexto_con_Numeros("Ingrese el nombre del producto:")
  precio = ingresaFloat("Ingrese el precio del producto:")
  nuevo_producto = pd.DataFrame({'Nombre': [nombre], 'Precio': [precio]})
  df_productos = pd.concat([df_productos, nuevo_producto], ignore_index=True)
  print('Producto ingresado correctamente.')

# Aqui se ingresan las facturas al archivo csv
def ingresar_factura():
  global df_facturas
  print(f"\nIngreso de Factura:")
  cliente = input("Ingrese el nombre del cliente: ")
  fecha = datetime.now() .strftime('%Y-%m-%d %H:%M:%S')
  productos_factura = []
  while True:
    nombre_producto = input("Ingrese el nombre del producto (o 'q' para salir):")
    if nombre_producto.lower() == "q":
      break
    productos_factura.append(nombre_producto)
  nueva_factura = pd.DataFrame({'Cliente': [cliente], 'Fecha': [fecha], 'Productos': [productos_factura]})
  df_facturas = pd.concat([df_facturas, nueva_factura], ignore_index=True)
  print('Factura ingresada correctamente.')

# Aqui se lee el archivo facturas y se cuentan las ventas y se suman los montos
def ver_resumen():
  global df_facturas, df_productos
  cantidad_ventas = len(df_facturas)
  monto_total_vendido = df_facturas.apply(lambda row: sum(df_productos[df_productos['Nombre'].isin(row['Productos'])]['Precio']), axis=1).sum()
  print(f"\nResunen:")
  print(f"Cantidad de ventas: {cantidad_ventas}")
  print(f"Monto total vendido: {monto_total_vendido}")

# Aqui se guarda el resumen
  opcion_guardar = input('¿Desea guardar el resumen? (s/n): ')
  if opcion_guardar.lower() == 's':
    archivo_guardar = ingresaTexto_con_Numeros('Ingrese el nombre del archivo para guardar el resumen: ')
    df_resumen = pd.DataFrame({'Cantidad_Ventas': [cantidad_ventas], 'Monto_Total_Vendido': [monto_total_vendido]})
    df_resumen.to_csv(archivo_guardar + '.csv', index=False)
    print(f"Resumen guardado en {archivo_guardar}.csv")

# Aqui se lee el archivo facturas y se cuentan las ventas por Cliente y Producto
def ver_detalle():
    global df_facturas
    print("\nDetalle:")
    print("1. Ventas por usuario")
    print("2. Ventas por producto")
    print("3. Ventas diaria:")
    print("4. Volver")

    opcion_detalle = ingresaEntero("Seleccione una opción: ")

    if opcion_detalle == 1:
      ventas_usuario = df_facturas.groupby('Cliente').size().reset_index(name='Cantidad_Ventas')
      print("\nVentas Por Usuario:")
      print(ventas_usuario)
    elif opcion_detalle == 2:
      ventas_producto = df_facturas.explode('Productos')
      ventas_producto = ventas_producto.groupby('Productos').size().reset_index(name='Cantidad_Ventas')
      print("\nVentas Por Producto:")
      print(ventas_producto)
    elif opcion_detalle == 3:
      ventas_diarias = df_facturas.groupby(df_facturas['Fecha'].str.split(' ').str[0]).size().reset_index(name='Cantidad_Ventas')
      print("\nVentas Diarias:")
      print(ventas_diarias)
    elif opcion_detalle == 4:
      return

# Aqui se guarda el detalle
    opcion_guardar = input("¿Desea guardar la vista? (s/n): ")
    if opcion_guardar.lower() == 's':
      archivo_guardar = ingresaTexto_con_Numeros("Ingrese el nombre del archivo para guardar la vista: ")
    if opcion_detalle == "1":
      ventas_usuario.to_csv(archivo_guardar + '.csv', index=False)
    elif opcion_detalle == '2':
      ventas_producto.to_csv(archivo_guardar + '.csv', index=False)
    elif opcion_detalle == '3':
      ventas_diarias.to_csv(archivo_guardar + '.csv', index=False)
      print(f"vista guardada en {archivo_guardar}.csv")

# Aqui se borran los datos
def borrar_datos():
  global df_productos, df_facturas
  print("\nBorrar Datos:")
  print("1. Borrar Productos")
  print("2. Borrar Facturas")
  print("3. Borrar Todos los Datos:")
  print("4. Volver")

  opcion_borrar = ingresaEntero("Seleccione una opcion: ")
  if opcion_borrar== 1:
    df_productos = pd.DataFrame(columns=['Nombre','Precio'])
    df_productos.to_csv(archivo_productos, index=False)
    print('Datos de Productos borrados.')
  elif opcion_borrar== 2:
    df_facturas = pd.DataFrame(columns=['Cliente','Fecha','Productos'])
    df_facturas.to_csv(archivo_facturas, index=False)
    print('Datos de Facturas borrados.')
  elif opcion_borrar== 3:
    df_productos = pd.DataFrame(columns=['Nombre','Precio'])
    df_facturas = pd.DataFrame(columns=['Cliente','Fecha','Productos'])
    print('Todos los Datos de han sido borrados.')
  elif opcion_borrar== 4:
    return

# Aqui se guardan los datos
def guardar_datos():
  global df_productos, df_facturas
  df_productos.to_csv(archivo_productos, index=False)
  df_facturas.to_csv(archivo_facturas, index=False)

def salir():
  print("\nSaliendo..")


Programa Principal

In [ ]:

def main():
    global df_productos, df_facturas
    while True:
      print("\nSistena de facturación y bodega")
      print("\nMenú Principal:")
      print("1. Ingresar producto")
      print("2. Ingresar factura")
      print("3. Ver resumen")
      print("4. Ver detalle")
      print("5. Borrar datos")
      print("6. Guardar datos")
      print("7. Salir")
      opcion = ingresaEntero("\nSeleccione una opción: ")

      if opcion == 1:
        ingresar_producto()
      elif opcion == 2:
        ingresar_factura()
      elif opcion == 3:
        ver_resumen()
      elif opcion == 4:
        ver_detalle()
      elif opcion == 5:
        borrar_datos()
      elif opcion == 6:
        guardar_datos()
      elif opcion == 7:
        salir()
        break
      else:
        print("Opción no válida. Inténtelo de nuevo.")

if __name__ == "__main__":
    main()

GIT

https://github.com/FelipexDiaz/cdd2/

Felipe Diaz